In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train_new.csv')
train = train.sample(n=10000).reset_index(drop=True)
train.head()

,image,class
0,v_Bowling_g23_c02.avi_frame1.jpg,Bowling
1,v_BabyCrawling_g18_c02.avi_frame6.jpg,BabyCrawling
2,v_PlayingDaf_g22_c05.avi_frame12.jpg,PlayingDaf
3,v_HorseRace_g18_c02.avi_frame5.jpg,HorseRace
4,v_Shotput_g23_c07.avi_frame1.jpg,Shotput


In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:43<00:00, 228.94it/s]


(10000, 224, 224, 3)

In [4]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [5]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [6]:
# creating the base model of pre-trained VGG16 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

219062272/219055592 [==============================] - 6s 0us/step


In [7]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8000, 5, 5, 1536)

In [8]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2000, 5, 5, 1536)

In [10]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8000, 5*5*1536)
X_test = X_test.reshape(2000, 5*5*1536)

In [11]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [12]:
# shape of images
X_train.shape

(8000, 38400)

In [13]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(38400,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

In [14]:
# defining a function to save the weights of best model
from tensorflow.keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('InceptionResNetV2weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [15]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [16]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)

Epoch 1/200
63/63 [==============================] - 9s 146ms/step - loss: 4.6407 - accuracy: 0.0190 - val_loss: 4.5427 - val_accuracy: 0.0570
Epoch 2/200
63/63 [==============================] - 9s 143ms/step - loss: 4.4281 - accuracy: 0.0465 - val_loss: 4.0969 - val_accuracy: 0.1210
Epoch 3/200
63/63 [==============================] - 9s 143ms/step - loss: 3.9796 - accuracy: 0.1126 - val_loss: 3.4719 - val_accuracy: 0.2265
Epoch 4/200
63/63 [==============================] - 9s 144ms/step - loss: 3.5422 - accuracy: 0.1744 - val_loss: 2.9858 - val_accuracy: 0.3035
Epoch 5/200
63/63 [==============================] - 9s 143ms/step - loss: 3.1100 - accuracy: 0.2501 - val_loss: 2.5189 - val_accuracy: 0.3975
Epoch 6/200
63/63 [==============================] - 9s 143ms/step - loss: 2.7552 - accuracy: 0.3104 - val_loss: 2.2213 - val_accuracy: 0.4480
Epoch 7/200
63/63 [==============================] - 9s 143ms/step - loss: 2.4647 - accuracy: 0.3641 - val_loss: 1.9178 - val_accuracy: 0.5065

Epoch 58/200
63/63 [==============================] - 8s 135ms/step - loss: 0.4516 - accuracy: 0.8633 - val_loss: 0.6331 - val_accuracy: 0.8540
Epoch 59/200
63/63 [==============================] - 9s 135ms/step - loss: 0.4251 - accuracy: 0.8714 - val_loss: 0.6280 - val_accuracy: 0.8595
Epoch 60/200
63/63 [==============================] - 8s 135ms/step - loss: 0.4423 - accuracy: 0.8675 - val_loss: 0.6119 - val_accuracy: 0.8655
Epoch 61/200
63/63 [==============================] - 8s 135ms/step - loss: 0.4204 - accuracy: 0.8701 - val_loss: 0.6143 - val_accuracy: 0.8670
Epoch 62/200
63/63 [==============================] - 9s 135ms/step - loss: 0.4333 - accuracy: 0.8726 - val_loss: 0.6093 - val_accuracy: 0.8630
Epoch 63/200
63/63 [==============================] - 8s 134ms/step - loss: 0.4400 - accuracy: 0.8677 - val_loss: 0.5971 - val_accuracy: 0.8715
Epoch 64/200
63/63 [==============================] - 9s 135ms/step - loss: 0.4375 - accuracy: 0.8730 - val_loss: 0.5979 - val_accuracy:

Epoch 115/200
63/63 [==============================] - 9s 136ms/step - loss: 0.3116 - accuracy: 0.9154 - val_loss: 0.5811 - val_accuracy: 0.8735
Epoch 116/200
63/63 [==============================] - 8s 134ms/step - loss: 0.2997 - accuracy: 0.9175 - val_loss: 0.6044 - val_accuracy: 0.8730
Epoch 117/200
63/63 [==============================] - 8s 134ms/step - loss: 0.2943 - accuracy: 0.9206 - val_loss: 0.6496 - val_accuracy: 0.8730
Epoch 118/200
63/63 [==============================] - 8s 134ms/step - loss: 0.2805 - accuracy: 0.9234 - val_loss: 0.6112 - val_accuracy: 0.8715
Epoch 119/200
63/63 [==============================] - 8s 135ms/step - loss: 0.2814 - accuracy: 0.9211 - val_loss: 0.6004 - val_accuracy: 0.8780
Epoch 120/200
63/63 [==============================] - 8s 134ms/step - loss: 0.2950 - accuracy: 0.9185 - val_loss: 0.5971 - val_accuracy: 0.8730
Epoch 121/200
63/63 [==============================] - 8s 134ms/step - loss: 0.3079 - accuracy: 0.9180 - val_loss: 0.5709 - val_ac

63/63 [==============================] - 9s 135ms/step - loss: 0.2427 - accuracy: 0.9377 - val_loss: 0.6210 - val_accuracy: 0.8765
Epoch 172/200
63/63 [==============================] - 9s 135ms/step - loss: 0.2243 - accuracy: 0.9400 - val_loss: 0.6769 - val_accuracy: 0.8750
Epoch 173/200
63/63 [==============================] - 9s 135ms/step - loss: 0.2297 - accuracy: 0.9380 - val_loss: 0.6431 - val_accuracy: 0.8805
Epoch 174/200
63/63 [==============================] - 9s 136ms/step - loss: 0.2532 - accuracy: 0.9342 - val_loss: 0.6588 - val_accuracy: 0.8770
Epoch 175/200
63/63 [==============================] - 9s 136ms/step - loss: 0.2661 - accuracy: 0.9334 - val_loss: 0.6410 - val_accuracy: 0.8695
Epoch 176/200
63/63 [==============================] - 9s 136ms/step - loss: 0.2631 - accuracy: 0.9315 - val_loss: 0.5947 - val_accuracy: 0.8745
Epoch 177/200
63/63 [==============================] - 9s 135ms/step - loss: 0.2508 - accuracy: 0.9314 - val_loss: 0.6371 - val_accuracy: 0.8755

In [17]:
# loading the trained weights
model.load_weights("InceptionResNetV2weight.hdf5")

In [18]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [19]:
# getting the test list
f = open("ucfTrainTestlist/testlist02.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi


In [20]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [21]:
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [23]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('UCF-101/'+videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 5*5*1536)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

  0%|                                                                                         | 0/3734 [00:00<?, ?it/s]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


100%|██████████████████████████████████████████████████████████████████████████████| 3734/3734 [35:00<00:00,  1.78it/s]


In [24]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

88.4841992501339